<a href="https://colab.research.google.com/github/fjgr/IA_BigData/blob/main/7RO/TAREA%205%20Q-LEARNING%208-PUZZLE/7RO_QLEARNING_8_PUZZLE_Francisco_Jose_Gonz%C3%A1lez_Rodriguez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Índice:
1. [Preparar el entorno y las dependencias](#preparar-entorno)
2. [Definir configuraciones y clases auxiliares](#definir-configuraciones)
3. [Implementar el entrenamiento](#implementar-entrenamiento)
4. [Resolver el problema](#resolver-problema)
5. [Ejecutar el programa principal](#ejecutar-programa-principal)


# 1.  Preparar el entorno y las dependencias
<a name="preparar-entorno"></a>

In [68]:
import os
import shutil
from collections import deque
import numpy as np
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass
import json
from datetime import datetime
from google.colab import files
from IPython.display import clear_output, display, HTML
import time
from tqdm.notebook import tqdm
import random

# 2.   Definir configuraciones y clases auxiliares
<a name="definir-configuraciones"></a>

In [69]:
# Clase de configuración y utilidades de visualización
@dataclass
class QLearningConfig:
    """
    Clase para almacenar parámetros de configuración para el algoritmo de Q-Learning.

    Atributos:
        - learning_rate: Tasa de aprendizaje para actualizar la tabla Q.
        - gamma: Factor de descuento para valorar recompensas futuras.
        - initial_epsilon: Probabilidad inicial de exploración.
        - epsilon_min: Límite inferior para epsilon durante el entrenamiento.
        - epsilon_decay: Factor de decaimiento de epsilon por episodio.
        - max_episodes: Número máximo de episodios de entrenamiento.
        - save_path: Ruta donde se guardan los datos entrenados.
        - checkpoint_interval: Episodios entre guardados de modelo.
    """
    learning_rate: float = 0.8
    gamma: float = 0.9
    initial_epsilon: float = 1.0
    epsilon_min: float = 0.1
    epsilon_decay: float = 0.995
    max_episodes: int = 50_000
    save_path: str = "/content/q_learning_data"
    checkpoint_interval: int = 5_000

def display_puzzle_state(state: str, title: str = ""):
    """
    Visualiza el estado actual del tablero del 8-puzzle en formato HTML.

    Args:
        state: Representación del tablero como una cadena de texto.
        title: Título opcional para mostrar junto al tablero.
    """
    html = f"""
    <div style="font-family: monospace; padding: 10px;">
        <h4 style="margin: 0;">{title}</h4>
        <div style="border: 2px solid black; display: inline-block; padding: 5px; margin-top: 5px;">
            <table style="border-collapse: collapse;">
    """

    # Construcción del tablero en formato HTML
    for i in range(0, 9, 3):
        html += "<tr>"
        for j in range(3):
            value = state[i + j]
            background = "#e6f3ff" if value == "9" else "white"
            html += f'<td style="border: 1px solid black; width: 30px; height: 30px; text-align: center; background-color: {background};">{value}</td>'
        html += "</tr>"

    html += """
            </table>
        </div>
    </div>
    """
    display(HTML(html))


# 3.   Implementar el entrenamiento
<a name="implementar-entrenamiento"></a>



In [70]:
# Clase para gestionar el entrenamiento del modelo
class EightPuzzleTrainer:
    """
    Clase para entrenar un modelo de Q-Learning que resuelve el problema del 8-puzzle.

    Atributos y métodos:
        - DIRECTIONS: Direcciones posibles del movimiento.
        - GOAL_STATE: Estado objetivo del rompecabezas.
        - Métodos para inicializar, entrenar y guardar/recuperar estados.
    """

    DIRECTIONS = [-3, 1, 3, -1]  # Arriba, Derecha, Abajo, Izquierda
    GOAL_STATE = "123456789"

    def __init__(self, config: QLearningConfig):
        """
        Inicializa el entrenador del modelo.

        Args:
            config: Parámetros de configuración para el entrenamiento.
        """
        self.config = config
        self.transitions: Dict[str, List[Optional[str]]] = {}
        self.q_table: Dict[str, Dict[int, float]] = {}
        self._ensure_save_directory()

    def _ensure_save_directory(self) -> None:
        """
        Crea el directorio de guardado si no existe.
        """
        if not os.path.exists(self.config.save_path):
            os.makedirs(self.config.save_path)

    @staticmethod
    def swap(state: str, i: int, j: int) -> str:
        """
        Intercambia dos posiciones en el estado del tablero.

        Args:
            state: Estado actual del tablero.
            i: Índice de la posición a intercambiar.
            j: Índice de la posición a intercambiar.

        Returns:
            Nuevo estado del tablero con los elementos intercambiados.
        """
        state_list = list(state)
        state_list[i], state_list[j] = state_list[j], state_list[i]
        return ''.join(state_list)

    def is_valid_move(self, index: int, direction: int) -> bool:
        """
        Verifica si un movimiento es válido dado el índice del espacio vacío y la dirección.

        Args:
            index: Índice de la posición vacía.
            direction: Dirección del movimiento.

        Returns:
            True si el movimiento es válido, False de lo contrario.
        """
        if direction == -3 and index < 3:  return False  # Arriba desde la primera fila
        if direction == 3 and index > 5:   return False  # Abajo desde la última fila
        if direction == -1 and index % 3 == 0: return False  # Izquierda desde la primera columna
        if direction == 1 and index % 3 == 2:  return False  # Derecha desde la última columna
        return True

    def generate_transitions(self, initial_state: str) -> None:
        """
        Genera todas las posibles transiciones desde un estado inicial utilizando BFS.

        Args:
            initial_state: Estado inicial del tablero.
        """
        visited = set()
        queue = deque([initial_state])

        while queue:
            current_state = queue.popleft()
            if current_state in visited:
                continue

            visited.add(current_state)
            empty_index = current_state.index('9')
            state_transitions = []

            for direction in self.DIRECTIONS:
                if self.is_valid_move(empty_index, direction):
                    new_state = self.swap(current_state, empty_index, empty_index + direction)
                    state_transitions.append(new_state)
                    if new_state not in visited:
                        queue.append(new_state)
                else:
                    state_transitions.append(None)

            self.transitions[current_state] = state_transitions

    def initialize_q_table(self) -> None:
        """
        Inicializa la tabla Q con ceros.
        """
        self.q_table = {
            state: {action: 0.0 for action in range(4)}
            for state in self.transitions
        }

    def calculate_reward(self, state: str, next_state: str, visited_states: set) -> float:
        """
        Calcula la recompensa basada en la distancia de Manhattan y penalizaciones.

        Args:
            state: Estado actual del tablero.
            next_state: Próximo estado del tablero.
            visited_states: Conjunto de estados visitados previamente.

        Returns:
            Valor de recompensa calculado.
        """
        def manhattan_distance(state: str) -> int:
            distance = 0
            for i, digit in enumerate(state):
                if digit == '9':
                    continue
                current_row, current_col = i // 3, i % 3
                goal_idx = int(digit) - 1
                goal_row, goal_col = goal_idx // 3, goal_idx % 3
                distance += abs(current_row - goal_row) + abs(current_col - goal_col)
            return distance

        if state == self.GOAL_STATE:
            return 1000

        current_distance = manhattan_distance(state)
        next_distance = manhattan_distance(next_state)

        distance_reward = (current_distance - next_distance) * 10
        loop_penalty = -50 if next_state in visited_states else 0
        progress_penalty = -5 if next_distance >= current_distance else 0

        return distance_reward + loop_penalty + progress_penalty

    def save_checkpoint(self, episode: int) -> None:
        """
        Guarda un punto de control del modelo.

        Args:
            episode: Número de episodio actual.
        """
        checkpoint_path = os.path.join(
            self.config.save_path,
            f'checkpoint_episode_{episode}.json'
        )
        data = {
            'episode': episode,
            'q_table': self.q_table,
            'transitions': self.transitions
        }
        with open(checkpoint_path, 'w') as f:
            json.dump(data, f)

    def load_latest_checkpoint(self) -> Optional[int]:
        """
        Carga el punto de control más reciente si existe.

        Returns:
            Número del último episodio si se carga correctamente, None de lo contrario.
        """
        checkpoints = [f for f in os.listdir(self.config.save_path)
                      if f.startswith('checkpoint_episode_')]
        if not checkpoints:
            return None

        latest = max(checkpoints)
        with open(os.path.join(self.config.save_path, latest), 'r') as f:
            data = json.load(f)
            self.q_table = data['q_table']
            self.transitions = data['transitions']
            return data['episode']

    def train(self) -> None:
        """
        Entrena el modelo de Q-Learning.
        """
        print("Generando transiciones...")
        self.generate_transitions(self.GOAL_STATE)
        print(f"Generados {len(self.transitions)} estados")

        print("Inicializando tabla Q...")
        self.initialize_q_table()

        epsilon = self.config.initial_epsilon
        states = list(self.transitions.keys())

        best_performance = float('-inf')
        no_improvement_count = 0

        print("Comenzando episodios de entrenamiento...")
        for episode in tqdm(range(self.config.max_episodes)):
            current_state = np.random.choice(states)
            visited_states = set()
            step_count = 0
            episode_reward = 0

            while step_count < 20:  # Máximo de 20 pasos por episodio
                visited_states.add(current_state)

                valid_actions = [
                    a for a in range(4)
                    if self.transitions[current_state][a] is not None
                ]

                if not valid_actions:
                    break

                if np.random.rand() < epsilon:
                    action = np.random.choice(valid_actions)
                else:
                    action = max(valid_actions,
                               key=lambda a: self.q_table[current_state][a])

                next_state = self.transitions[current_state][action]
                reward = self.calculate_reward(current_state, next_state, visited_states)
                episode_reward += reward

                self.q_table[current_state][action] = (
                    (1 - self.config.learning_rate) * self.q_table[current_state][action] +
                    self.config.learning_rate * (
                        reward +
                        self.config.gamma * max(self.q_table[next_state].values())
                    )
                )

                if next_state == self.GOAL_STATE:
                    break

                current_state = next_state
                step_count += 1

            if episode_reward > best_performance:
                best_performance = episode_reward
                no_improvement_count = 0
            else:
                no_improvement_count += 1

            if no_improvement_count > 1000:
                print(f"\nTerminación temprana en el episodio {episode} debido a falta de mejoras")
                break

            epsilon = max(
                self.config.epsilon_min,
                epsilon * self.config.epsilon_decay
            )

            if episode > 0 and episode % self.config.checkpoint_interval == 0:
                self.save_checkpoint(episode)
                print(f"\nPunto de control guardado en el episodio {episode}")
                print(f"Epsilon actual: {epsilon:.4f}")

        print("Entrenamiento completado!")

# 4.  Resolver el problema
<a name="resolver-problema"></a>

In [63]:
# Clase para resolver el puzzle con el modelo entrenado
class EightPuzzleSolver:
    """
    Clase de solucionador mejorada con búsqueda de ruta mejorada y prevención de bucles
    """
    def __init__(self, trainer: EightPuzzleTrainer):
        self.trainer = trainer
        self.move_names = {-3: "UP", 1: "RIGHT", 3: "DOWN", -1: "LEFT"}

    def get_manhattan_distance(self, state: str) -> int:
        """
        Calcula la distancia de Manhattan entre el estado actual y el estado objetivo.
        Esta métrica suma las distancias de cada número a su posición correcta.

        Args:
            state: Estado actual del puzzle como cadena.

        Returns:
            Distancia total de Manhattan.
        """
        distance = 0
        for i, digit in enumerate(state):
            if digit != '9':
                current_row, current_col = i // 3, i % 3
                goal_idx = int(digit) - 1
                goal_row, goal_col = goal_idx // 3, goal_idx % 3
                distance += abs(current_row - goal_row) + abs(current_col - goal_col)
        return distance

    def evaluate_move(self, state: str, action: int, visited_states: set) -> float:
        """
        Evalúa la calidad de un movimiento considerando múltiples factores:
        - Valor Q del entrenamiento
        - Penalización por estados ya visitados
        - Mejora en la distancia al objetivo
        - Bonificación por movimientos no intentados recientemente
        """
        next_state = self.trainer.transitions[state][action]
        if next_state is None:
            return float('-inf')

        # Base Q-value from training
        q_value = self.trainer.q_table[state][action]

        # Penalties and bonuses
        visit_penalty = -100 if next_state in visited_states else 0
        current_distance = self.get_manhattan_distance(state)
        next_distance = self.get_manhattan_distance(next_state)
        distance_improvement = (current_distance - next_distance) * 20

        # Bonus for moves that haven't been tried recently
        recency_bonus = 0 if next_state in visited_states else 50

        return q_value + visit_penalty + distance_improvement + recency_bonus

    def solve(self, initial_state: str, max_steps: int = 1000) -> List[str]:
        """
        Resuelve el puzzle utilizando una estrategia mejorada de selección de movimientos
        y prevención de bucles.

        Características:
        - Detecta cuando se queda atascado y aumenta la exploración
        - Visualiza cada paso con información detallada
        - Considera la distancia Manhattan para evaluar el progreso
        - Implementa mecanismos para evitar bucles
        """
        if not self.trainer.q_table:
            raise ValueError("No hay ningún modelo entrenado disponible")

        if not self.is_solvable(initial_state):
            print("Advertencia: ¡Esta configuración de rompecabezas no tiene solución!")
            return []

        path = []
        visited_states = set()
        current_state = initial_state
        stuck_counter = 0
        last_distance = float('inf')

        print("\nEstado inicial:")
        display_puzzle_state(current_state, "Posición inicial")
        time.sleep(1)

        for step in range(max_steps):
            path.append(current_state)
            visited_states.add(current_state)

            if current_state == self.trainer.GOAL_STATE:
                print(f"\nSolución encontrada en {step} pasos!")
                display_puzzle_state(current_state, f"¡Objetivo alcanzado! (Pasos {step})")
                return path

            # Get all valid actions and their evaluations
            valid_actions = []
            action_values = {}

            for action in range(4):
                next_state = self.trainer.transitions[current_state][action]
                if next_state is not None:
                    move_value = self.evaluate_move(current_state, action, visited_states)
                    action_values[action] = move_value
                    valid_actions.append(action)

            if not valid_actions:
                print("No valid moves available")
                break

            # Select best action considering various factors
            current_distance = self.get_manhattan_distance(current_state)

            # Check if we're stuck
            if current_distance >= last_distance:
                stuck_counter += 1
            else:
                stuck_counter = 0
            last_distance = current_distance

            # If stuck, increase exploration
            if stuck_counter > 3:
                # Choose a random action that leads to an unvisited state
                unvisited_actions = [a for a in valid_actions
                                   if self.trainer.transitions[current_state][a] not in visited_states]
                if unvisited_actions:
                    action = random.choice(unvisited_actions)
                else:
                    action = random.choice(valid_actions)
                stuck_counter = 0
            else:
                # Choose the best action based on evaluations
                action = max(action_values.keys(), key=lambda a: action_values[a])

            next_state = self.trainer.transitions[current_state][action]
            move_name = self.move_names[self.trainer.DIRECTIONS[action]]

            clear_output(wait=True)
            print(f"\nPaso {step + 1}:")
            print(f"Movimiento: {move_name}")
            print(f"Manhattan distancia a la meta: {current_distance}")
            display_puzzle_state(current_state, f"Estado actual (anterior {move_name})")
            time.sleep(0.5)

            current_state = next_state

        print("\nNo se pudo resolver dentro del límite de pasos")
        return path

    @staticmethod
    def is_solvable(state: str) -> bool:
        """Determinar si el estado del rompecabezas es solucionable"""
        numbers = [int(x) for x in state if x != '9']
        inversions = sum(1 for i in range(len(numbers))
                        for j in range(i + 1, len(numbers))
                        if numbers[i] > numbers[j])
        return inversions % 2 == 0

    @staticmethod
    def generate_random_initial_state() -> str:
        """Generar un estado inicial aleatorio solucionable"""
        while True:
            state = ''.join(random.sample("123456789", 9))
            if EightPuzzleSolver.is_solvable(state):
                return state

def main():
    """
    Función principal para configurar el entorno, entrenar el modelo y resolver el puzzle.
    """
    print("Configurando entorno de entrenamiento...")
    config = QLearningConfig()
    trainer = EightPuzzleTrainer(config)

    print("\nGenerando estado inicial aleatorio válido...")
    # Call generate_random_initial_state from EightPuzzleSolver instead of EightPuzzleTrainer
    # Corrected line: Calling is_solvable from EightPuzzleSolver
    initial_state = EightPuzzleSolver.generate_random_initial_state()
    print(f"Estado inicial generado: {initial_state}")

    print("\nIniciando proceso de entrenamiento...")
    trainer.train()

    print("\nEntrenamiento completado. Iniciando solución del puzzle...")
    solver = EightPuzzleSolver(trainer)

    print("\nResolviendo el puzzle...")
    solution_path = solver.solve(initial_state)

    print("\nEstadísticas de la solución:")
    print(f"Movimientos totales: {len(solution_path) - 1}")
    print(f"Estados únicos visitados: {len(set(solution_path))}")
    print(f"Estado objetivo alcanzado: {'Sí' if solution_path[-1] == trainer.GOAL_STATE else 'No'}")

# 5.  Ejecutar el programa principal
<a name="ejecutar-programa-principal"></a>

In [71]:
# Run the main function
main()


Step 100:
Move: UP
Manhattan distance to goal: 9


4,3,2
1,8,9
5,7,6



Could not solve within step limit

Estadísticas de la solución:
Movimientos totales: 99
Estados únicos visitados: 99
Estado objetivo alcanzado: No
